## 서울특별시 지형 데이터 
출처 : 국가공간정보포털 - 토지특성정보  
http://openapi.nsdi.go.kr/nsdi/eios/ServiceDetail.do?svcSe=F&svcId=F024

#### 작업중인 단계
1. 지형데이터의 table schema와 기준 이해하기 : 완료 => Notion
2. 사용할 최신 데이터만 선별하기
3. 집계구별로 묶기 - 우리 데이터랑 매칭 시키기
    - 우리 데이터에서 가진 집계구 코드와 일치하는 곳들을 merge해서 사용함
4. 가용 정보 선정
    - 용도 지역명1
    - 용도 지역명2
    - 지형 높이
    - 도로 접면
    - 공시 지가 평균값 / 확률 / 
    - 토지 이용 상황

In [1]:
import pandas as pd
import numpy as np
import pandas_gbq as pd_gbq
import datetime

In [2]:
df = pd.read_csv('seoul_topology.csv', encoding = 'cp949')

/Users/yeonjin/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.tail()

,고유번호,법정동코드,법정동명,대장구분코드,대장구분명,지번,토지일련번호,기준년도,기준월,지목코드,...,토지이용상황코드,토지이동상황,지형높이코드,지형높이,지형형상코드,지형형상,도로접면코드,도로접면,공시지가,데이터기준일자
6558513,1174011000307390000,1174011000,서울특별시 강동구 강일동,3.0,가지번,739,29238,2014,1,22,...,930,공원등,0,지정되지않음,0,지정되지않음,0,지정되지않음,1257000,2017-10-17
6558514,1174011000307390000,1174011000,서울특별시 강동구 강일동,3.0,가지번,739,29231,2015,1,22,...,930,공원등,0,지정되지않음,0,지정되지않음,0,지정되지않음,874500,2017-10-17
6558515,1174011000307400000,1174011000,서울특별시 강동구 강일동,3.0,가지번,740,29232,2013,1,8,...,140,아파트,2,평지,7,부정형,2,광대소각,4000000,2017-10-17
6558516,1174011000307400000,1174011000,서울특별시 강동구 강일동,3.0,가지번,740,29239,2014,1,8,...,140,아파트,2,평지,7,부정형,4,중로한면,4175000,2017-10-17
6558517,1174011000307400000,1174011000,서울특별시 강동구 강일동,3.0,가지번,740,29232,2015,1,8,...,140,아파트,2,평지,7,부정형,6,소로한면,3970000,2017-10-17


In [5]:
df.columns

Index(['고유번호', '법정동코드', '법정동명', '대장구분코드', '대장구분명', '지번', '토지일련번호', '기준년도',
       '기준월', '지목코드', '지목명', '토지면적', '용도지역코드1', '용도지역명1', '용도지역코드2', '용도지역명2',
       '토지이용상황코드', '토지이동상황', '지형높이코드', '지형높이', '지형형상코드', '지형형상', '도로접면코드',
       '도로접면', '공시지가', '데이터기준일자'],
      dtype='object')

In [22]:
len(df)

6558518

In [47]:
df.dtypes

고유번호         object
법정동코드         int64
법정동명         object
대장구분코드      float64
대장구분명        object
지번           object
토지일련번호        int64
기준년도          int64
기준월           int64
지목코드          int64
지목명          object
토지면적        float64
용도지역코드1       int64
용도지역명1       object
용도지역코드2       int64
용도지역명2       object
토지이용상황코드      int64
토지이동상황       object
지형높이코드        int64
지형높이         object
지형형상코드        int64
지형형상         object
도로접면코드        int64
도로접면         object
공시지가          int64
데이터기준일자      object
dtype: object

## '고유번호'를 기준으로 각 필지에서 산출된 최신 데이터만 선별해야함
* 고유번호 = 필지를 기반으로 부여됨
* 필지 : 국토 등록 기초 단위
    * 필지(筆地) : 필(筆)은 구획된 논이나 밭, 임야, 대지 따위를 세는 단위이다. 땅에 대한 소유권이나 건물이 앉은 터를 기준으로 해서, 토지 구역 경계로 갈라 정한 국토를 등록하는 기본단위이다. 필지는 도시에서 용도지역, 용도지구를 구분하는 도시계획에서 기초 단위이다. 개별 필지는 지번과 지목을 가지고 있고, 도시지역에 있는 필지는 토지이용계획이 부여된다.

In [82]:
df[df['고유번호'] == 1111010100100010000]
# 하나의 고유번호에 [기준년도, 기준월, 데이터기준일자]에 따라 다른 데이터가 존재함

,고유번호,법정동코드,법정동명,대장구분코드,대장구분명,지번,토지일련번호,기준년도,기준월,지목코드,...,토지이용상황코드,토지이동상황,지형높이코드,지형높이,지형형상코드,지형형상,도로접면코드,도로접면,공시지가,데이터기준일자
0,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2013,1,8,...,120,연립,2,평지,4,사다리형,6,소로한면,2840000,2017-10-17
1,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2014,1,8,...,140,아파트,2,평지,4,사다리형,6,소로한면,2933000,2017-10-17
2,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2015,1,8,...,140,아파트,2,평지,4,사다리형,6,소로한면,3053000,2017-10-17
3,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2016,1,8,...,140,아파트,2,평지,4,사다리형,6,소로한면,3234000,2017-10-17
4,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2017,1,8,...,140,아파트,2,평지,4,사다리형,6,소로한면,3488000,2018-05-23
5,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2018,1,8,...,140,아파트,2,평지,4,사다리형,6,소로한면,3765000,2018-07-31
6,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2019,1,8,...,140,아파트,2,평지,4,사다리형,6,소로한면,3961000,2019-08-08


## 어떤 기준으로 최대값을 찾을까?
1. 기준년도
2. 데이터기준일자
3. datetime.date(기준년도, 기준월 ,1)로 기준일자 제작 후 사용
4. 3의 기준일자와 데이터기준일자 중첩

In [34]:
# duplicate를 없앤 고유번호의 수
len(df['고유번호'].drop_duplicates())

985465

In [84]:
# 1 
df.groupby('기준년도')['고유번호'].count()

기준년도
2013    954102
2014    946648
2015    940631
2016    939576
2017    930804
2018    926366
2019    920391
Name: 고유번호, dtype: int64

In [103]:
len(df[df.groupby(['고유번호'])['기준년도'].transform(max) == df['기준년도']])

987584

In [85]:
# 2 데이터 기준일자 활용
df['데이터기준일자'] = pd.to_datetime(df['데이터기준일자'])

In [97]:
# case 1
len(pd.DataFrame(df.groupby('고유번호')['데이터기준일자'].max()))

985465

In [105]:
# case 2
len(df[df.groupby(['고유번호'])['데이터기준일자'].transform(max) == df['데이터기준일자']])

1059179

In [114]:
df.groupby(['고유번호'])['데이터기준일자'].transform(max)

0         2019-08-08
1         2019-08-08
2         2019-08-08
3         2019-08-08
4         2019-08-08
             ...    
6558513   2017-10-17
6558514   2017-10-17
6558515   2017-10-17
6558516   2017-10-17
6558517   2017-10-17
Name: 데이터기준일자, Length: 6558518, dtype: datetime64[ns]

### 데이터 기준일자의 max값 기준으로 데이터를 선별할 때 차이가 발생하는 이유

* case 1 : 고유번호를 기준으로 '데이터기준일자' 중 최대값에 해당하는 값을 보여줌.
* case 2 : 
    1. 각 고유번호 내에서 <최대인 데이터기준일자 x>를 찾음 
    2. 각 row의 고유번호에 매칭되는 x 값을 라벨링해줌 (df 길이만큼, 일종의 컬럼 형성)
    3. 해당 값과 df['기준일자']가 동일한지 확인  

고로, 하나의 고유번호일지라도 '데이터기준일자'가 최대인 데이터를 여러개 보유하고 있기 때문에 그런 경우의 수만큼 데이터의 길이가 길어짐  

* 대안 : drop_duplicate( keep = last )를 할 수 있음. 그러나, id 중 일부는 string값과 함께 들어가있어서 sort를 하기 까다로운 상황
* 결론 : 3번째 기준년월으로 '기준일자' 컬럼을 제작하고 해보기로 결정.

In [112]:
# 3. 기준일자 생성 후 비교
##1. 기준년도, 기준일을 datetime 객체로 바꿔주기
LYM = []
# lable_for_year_month
for i in range(len(df)):
    LYM.append(datetime.date(df.iloc[i,7], df.iloc[i,8],1))

In [100]:
df['standard_date'] = LYM

In [111]:
## 2. 데이터 기준일자를 기준으로 체크 -> OK! 
len(df[df.groupby(['고유번호'])['standard_date'].transform(max) == df['standard_date']])

985465

In [115]:
ndf = df[df.groupby(['고유번호'])['standard_date'].transform(max) == df['standard_date']]

In [117]:
ndf.head()

,고유번호,법정동코드,법정동명,대장구분코드,대장구분명,지번,토지일련번호,기준년도,기준월,지목코드,...,토지이동상황,지형높이코드,지형높이,지형형상코드,지형형상,도로접면코드,도로접면,공시지가,데이터기준일자,standard_date
6,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2019,1,8,...,아파트,2,평지,4,사다리형,6,소로한면,3961000,2019-08-08,2019-01-01
13,1111010100100010001,1111010100,서울특별시 종로구 청운동,1.0,일반,1-1,2,2019,1,14,...,도로등,2,평지,7,부정형,0,지정되지않음,1296000,2019-08-08,2019-01-01
20,1111010100100010002,1111010100,서울특별시 종로구 청운동,1.0,일반,1-2,3,2019,1,8,...,주거나지,3,완경사,7,부정형,12,맹지,3267000,2019-08-08,2019-01-01
27,1111010100100010003,1111010100,서울특별시 종로구 청운동,1.0,일반,1-3,4,2019,1,8,...,연립,2,평지,7,부정형,6,소로한면,4255000,2019-08-08,2019-01-01
34,1111010100100010004,1111010100,서울특별시 종로구 청운동,1.0,일반,1-4,5,2019,1,14,...,도로등,2,평지,7,부정형,0,지정되지않음,1524000,2019-08-08,2019-01-01


In [118]:
#ndf.to_csv('tfseoul_1.csv', index = False)